<a href="https://colab.research.google.com/drive/1JWFwU2s-KdN0G1epAMaufKANo6fmFoav?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

###	📄 What is HyDE?

Hypothetical Document Embeddings (HyDE) is an advance RAG technique that generates synthetic document embeddings based on the query, rather than relying solely on existing document embeddings. This approach creates a "hypothetical" perfect document that would ideally answer the query, and then uses this embedding to retrieve the most similar actual documents from the database. HyDE aims to bridge the gap between query intent and document content, especially for complex or nuanced queries.

### 🔧 HyDE-RAG Implementation:

1. **Hypothetical Document Generation:** We use the LLM to generate a hypothetical passage that would answer the given query.
2. **Retrieval using Hypothetical Document:** We use the generated hypothetical document to retrieve relevant passages from our vector store.
3. **Final Response Generation:** Using the retrieved context, we generate a final response to the original query.

# ⚙️ Setup

1. **[LLM](https://groq.com/):** Groq's free Open source LLM endpoints([Groq API Key](https://console.groq.com/keys))
2. **[Vector Store](https://www.pinecone.io/learn/vector-database/):** [ChromaDB](https://www.trychroma.com/)
3. **[Embedding Model](https://qdrant.tech/articles/what-are-embeddings/):** [nomic-embed-text-v1.5](https://www.nomic.ai/blog/posts/nomic-embed-text-v1)
4. **[LLM Framework](https://python.langchain.com/v0.2/docs/introduction/):** LangChain
5. **[Huggingface API Key](https://huggingface.co/settings/tokens)**





### Install required libraries

In [1]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.3.19 \
     langchain-groq==0.2.4 \
     langchain-chroma==0.2.2 \
     langchain-community==0.3.18 \
     langchain-huggingface==0.1.2 \
     einops==0.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.1/414.1 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.2 MB/s eta 

### Import related libraries related to Langchain, HuggingfaceEmbedding

In [2]:
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import WebBaseLoader

In [3]:
import os
import getpass

#### Provide a Groq API key. You can create one to access free open-source models at the following link.

[Groq API Creation Link](https://console.groq.com/keys)




In [4]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


### Provide Huggingface API Key. You can create Huggingface API key at following link

[Huggingface API Creation Link](https://huggingface.co/settings/tokens)




In [5]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


### Step 1: Load and preprocess data code

In [6]:
def load_and_process_data(url):
    # Load data from web
    loader = WebBaseLoader(url)
    data = loader.load()

    # Split text into chunks (Experiment with Chunk Size and Chunk Overlap to get optimal chunking)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(data)

    return chunks

## Step 2: Create vector store code

In [7]:
def create_vector_store(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})
    vectorstore = Chroma.from_documents(chunks, embeddings)
    return vectorstore

### Step 3: HyDE-RAG related code

1. **Hypothetical Document Generation:** We use the LLM to generate a hypothetical passage that would answer the given query.
2. **Retrieval using Hypothetical Document:** We use the generated hypothetical document to retrieve relevant passages from our vector store.
3. **Final Response Generation:** Using the retrieved context, we generate a final response to the original query.

In [8]:
def hyde_rag(query, vectorstore, llm):
    # Generate hypothetical document
    hyde_prompt = ChatPromptTemplate.from_template(
        "Given the following question, generate a hypothetical passage that would answer this question:\nQuestion: {query}\nHypothetical Passage:"
    )
    hyde_chain = hyde_prompt | llm
    hypothetical_doc = hyde_chain.invoke({"query": query})

    # Retrieve relevant documents using the hypothetical document
    retrieved_docs = vectorstore.similarity_search(hypothetical_doc.content, k=3)
    context = "\n".join([doc.page_content for doc in retrieved_docs])

    # Generate final response
    final_prompt = ChatPromptTemplate.from_template(
        "Based on the following context, please answer the question:\nContext: {context}\nQuestion: {query}\nAnswer:"
    )
    final_chain = final_prompt | llm
    final_response = final_chain.invoke({"context": context, "query": query})

    return {
        "hypothetical_document": hypothetical_doc.content,
        "retrieved_context": context,
        "final_answer": final_response.content
    }

### Step 4: Create chunk of web data to Chroma Vector Store

In [9]:
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.5
)

# Load and process data
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
chunks = load_and_process_data(url)

# Create vector store
vectorstore = create_vector_store(chunks)

/usr/local/lib/python3.11/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

### Step 5: Run HyDE-RAG

This implementation shows the key parts of HyDE-RAG:

1. Generation of a hypothetical document based on the query
2. Using the hypothetical document for retrieval instead of the original query
3. Final response generation using the retrieved context

In [10]:
# Example query
query = "What are the ethical considerations in AI development?"
result = hyde_rag(query, vectorstore, llm)

print("Hypothetical Document:")
print(result["hypothetical_document"])
print("\nRetrieved Context:")
print(result["retrieved_context"])
print("\nFinal Answer:")
print(result["final_answer"])

Hypothetical Document:
As artificial intelligence (AI) continues to transform industries and revolutionize the way we live, it is imperative to consider the ethical implications of its development. The creation of AI systems that can learn, reason, and interact with humans raises a multitude of ethical concerns that must be addressed.

One of the primary ethical considerations in AI development is the potential for bias and discrimination. AI systems are only as good as the data they are trained on, and if that data is biased or discriminatory, the AI system will likely reflect those biases. For example, if an AI system is trained on a dataset that contains more male than female names, it may be more likely to recognize and respond to male names, potentially perpetuating gender-based discrimination. Similarly, AI systems trained on biased data may disproportionately affect certain groups, such as people of color, in terms of job opportunities, healthcare, and other areas.

Another ethi